# 深度卷积模型：案例分析

## 1. 案例分析

### 1.1 为什么要进行案例分析

过去几年来，计算机视觉领域的主要工作，就在研究如何拼装卷积层、池化层、全连接层等等卷积神经网络模型的基本组件，组合出较为有效的模型，有效的模型通常可以在不同的计算机视觉模型之间通用。因此，通过学习这些模型实例，可以对深度卷积模型有更好的理解。

### 1.2 经典网络

LeNet-5，用来处理灰度图片的数字识别问题。
    - 在LeNet-5的年代，补全的技巧还不常用，所有卷积层都不补全
    - 池化层在当时更多采用的是平均池化，目前最大池化更为常见
    - 模型大约有60000个参数，相对于现代神经网络，规模较小
    - 随着模型的深入，高度和宽度不断减小，通道数不断增加，这个实践沿用至今
    - 当时的激活函数采用的是sigmoid或者tanh，还没有使用ReLU
    - 当时单个过滤器，并不会处理所有的通道，这点和现在也不一样
    - 当时在池化层后面会接一个非线性的激活层，和现在不同

![LeNet-5.png](img/LeNet-5.png)

AlexNet，真正使得计算机视觉领域开始重视神经网络
    - 已经开始使用ReLU作为激活函数
    - 大约6000万的参数，中等规模
    - 当时的GPU计算能力还不够强，论文里涉及两个GPU的通信
    - 用到了Local Response Normalization(LRN)的技巧，这个技巧目前已经不常用了
    - same padding，同一补全

![AlexNet.png](img/AlexNet.png)

VGG-16，模式更为明确
    - 固定3×3的过滤器，步长s=1，同一补全；最大池化层2×2，步长s=2
    - 每次卷积层（连续两层）后，通道数翻倍
    - 每次池化层后，高度和宽度减半
    - 大约有1.38亿的参数，现代规模的神经网络
    - VGG-16中的16是值整个神经网路中共有16层带有参数的层，另外还有一个更大的VGG-19版本，不常用

![VGG-16.png](img/VGG-16.png)

### 1.3 残差网络 ResNets

由于梯度消失和梯度爆炸问题，训练非常大的神经网络通常非常困难。而残差网络解决了这个问题。残差网络的基本组成结构如下：
![Residual block.png](img/Residual block.png)

由于计算能力的问题，普通神经网络（Plain Network）在实践中随着层数增多，并不能获得更好的效果。而残差神经网络使训练层数非常多的模型成为可能。
![Residual Network.png](img/Residual Network.png)